In [1]:
from pyomo.environ import *
import os
import idaes
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import math
from idaes.core.util.math import safe_log
from idaes.core.util.model_statistics import degrees_of_freedom
from Initialization import Initialization

In [2]:
# Parameters
component_list = ["benzene","toluene"]
phase_list = ["Liq", "Vap"]
pressure_crit_data = {"benzene": 48.9e5, "toluene": 41.0e5}
temperature_crit_data = {"benzene": 562.2, "toluene": 591.8}
kappa_data = {
            ("benzene", "benzene"): 0.0000,
            ("benzene", "toluene"): 0.0000,
            ("toluene", "benzene"): 0.0000,
            ("toluene", "toluene"): 0.0000,
        }
omega_data = {"benzene": 0.212, "toluene": 0.263}
R = 8.314462618

In [3]:
m = ConcreteModel()
# define external function for finding cubic roots
m.crl = ExternalFunction(
    library=os.path.join(idaes.bin_directory, "cubic_roots.so"),
    function="cubic_root_l_ext",
)
m.crh = ExternalFunction(
    library=os.path.join(idaes.bin_directory, "cubic_roots.so"),
    function="cubic_root_h_ext",
)

m.Comp = Set(initialize=component_list)
m.Phase = Set(initialize=phase_list)

m.T = Var(initialize = 298.15)
m.P = Var(initialize = 101325, units=units.Pa)

m.temperature_bubble = Var(initialize=400)
m._mole_frac_tbub = Var(m.Comp,initialize=1 / len(m.Comp),bounds=(0, None))
m.temperature_dew = Var(initialize=400)
m._mole_frac_tdew = Var(m.Comp,initialize=1 / len(m.Comp),bounds=(0, None))

m.Tc = Param(m.Comp,initialize = temperature_crit_data)
m.Pc = Param(m.Comp,initialize = pressure_crit_data)
m.kappa = Param(m.Comp, m.Comp, initialize = kappa_data)
m.omega = Param(m.Comp, initialize = omega_data)

# for SRK cEOS
m.OmegaA = Param(default=0.45724)
m.OmegaB = Param(default=0.07780)
m.EoS_u = Param(default=2)
m.EoS_w = Param(default=-1)
m.EoS_p = sqrt(m.EoS_u**2 - 4 * m.EoS_w)
def func_fw(m,j):
    return 0.37464 + 1.54226 * m.omega[j] - 0.26992 * m.omega[j] ** 2
m.fw = Expression(m.Comp, rule = func_fw)

m.antoine_coeff_A = Param(m.Comp,mutable=False,
                          initialize={"benzene": 4.202, "toluene": 4.216})

m.antoine_coeff_B = Param(m.Comp,mutable=False,
                          initialize={"benzene": 1322, "toluene": 1435})

m.antoine_coeff_C = Param(m.Comp,mutable=False,
                          initialize={"benzene": -38.56, "toluene": -43.33})

m.flow_mol = Var(initialize=1.0,domain=NonNegativeReals) # F
m.mole_frac_comp = Var(m.Comp, bounds=(0, None), initialize=0.5) # z
m.flow_mol_phase = Var(m.Phase, initialize=0.5, domain=NonNegativeReals) # L, V
m.mole_frac_phase_comp = Var(m.Phase, m.Comp, initialize=0.5,bounds=(0, None)) # x, y

def rule_total_mass_balance(m): # L + V = F
    return m.flow_mol_phase["Liq"] + m.flow_mol_phase["Vap"] == m.flow_mol
m.total_flow_balance = Constraint(rule=rule_total_mass_balance)

def rule_comp_mass_balance(m, c): # L*x + V*y = F*z
    return (
        m.flow_mol * m.mole_frac_comp[c]
        == m.flow_mol_phase["Liq"] * m.mole_frac_phase_comp["Liq", c]
        + m.flow_mol_phase["Vap"] * m.mole_frac_phase_comp["Vap", c]
    )
m.component_flow_balances = Constraint(m.Comp, rule=rule_comp_mass_balance)

def rule_mole_frac(m): # sum(x) - sum(y) = 1
    return (
        sum(m.mole_frac_phase_comp["Liq", c] for c in m.Comp)
        - sum(m.mole_frac_phase_comp["Vap", c] for c in m.Comp)
        == 0
    )
m.sum_mole_frac = Constraint(rule=rule_mole_frac)

In [4]:
# 用逸度估算泡露点时，需要分别计算 泡点/露点温度下 的 气相和液相的逸度
#-----------------------液相泡点温度---------------------------
def _bubble_temp_liq(m, j):
    def a(k):
        return (
            m.OmegaA * ((R * m.Tc[k]) ** 2 / m.Pc[k])
            * ((1 + m.fw[k] * (1 - sqrt(m.temperature_bubble / m.Tc[k]))) ** 2)
        )

    def b(k):
        return (m.OmegaB * R * m.Tc[k] / m.Pc[k])
        
    am1 = sum(
        sum(
            m.mole_frac_comp[i]
            * m.mole_frac_comp[j]
            * sqrt(a(i) * a(j))
            * (1 - m.kappa[i, j])
            for j in m.Comp
        )
        for i in m.Comp
    )
    bm1 = sum(m.mole_frac_comp[i] * b(i) for i in m.Comp)

    A1 = am1 * m.P / (R * m.temperature_bubble) ** 2
    B1 = bm1 * m.P / (R * m.temperature_bubble)

    delta = (
        2 * sqrt(a(j)) / am1 * sum(
            m.mole_frac_comp[i] * sqrt(a(i)) * (1 - m.kappa[j, i])
            for i in m.Comp
        )
    )

    Z1 = m.crl.evaluate(args=((-(1+B1-m.EoS_u*B1)), 
                             (A1-m.EoS_u*B1-(m.EoS_u-m.EoS_w)*B1**2), 
                             (-A1*B1-m.EoS_w*B1**2-m.EoS_w*B1**3)))

    return exp(
        (
            b(j) / bm1 * (Z1 - 1) * (B1 * m.EoS_p)
            - safe_log(Z1 - B1, eps=1e-6) * (B1 * m.EoS_p)
            + A1
            * (b(j) / bm1 - delta)
            * safe_log(
                (2 * Z1 + B1 * (m.EoS_u + m.EoS_p))
                / (2 * Z1 + B1 * (m.EoS_u - m.EoS_p)),
                eps=1e-6,
            )
        )
        / (B1 * m.EoS_p)
    )
m.bubble_temp_liq = Expression(m.Comp, rule = _bubble_temp_liq)

#-----------------------液相露点温度---------------------------
def _dew_temp_liq(m, j):
    def a(k):
        return (
            m.OmegaA * ((R * m.Tc[k]) ** 2 / m.Pc[k])
            * ((1 + m.fw[k] * (1 - sqrt(m.temperature_dew / m.Tc[k]))) ** 2)
        )
    def b(k):
        return (m.OmegaB * R * m.Tc[k] / m.Pc[k])
        
    am = sum(
        sum(
            m._mole_frac_tdew[i]
            * m._mole_frac_tdew[j]
            * sqrt(a(i) * a(j))
            * (1 - m.kappa[i, j])
            for j in m.Comp
        )
        for i in m.Comp
    )
    bm = sum(m._mole_frac_tdew[i] * b(i) for i in m.Comp)
    
    A = am * m.P / (R * m.temperature_dew) ** 2
    B = bm * m.P / (R * m.temperature_dew)
    
    delta = (
        2 * sqrt(a(j)) / am
        * sum(m._mole_frac_tdew[i] * sqrt(a(i)) * (1 - m.kappa[j, i]) for i in m.Comp
        )
    )
    
    Z = m.crl.evaluate(args=((-(1+B-m.EoS_u*B)), 
                             (A-m.EoS_u*B-(m.EoS_u-m.EoS_w)*B**2), 
                             (-A*B-m.EoS_w*B**2-m.EoS_w*B**3)))

    return exp(
            (
                b(j) / bm * (Z - 1) * (B * m.EoS_p)
                - safe_log(Z - B, eps=1e-6) * (B * m.EoS_p)
                + A
                * (b(j) / bm - delta)
                * safe_log(
                    (2 * Z + B * (m.EoS_u + m.EoS_p))
                    / (2 * Z + B * (m.EoS_u - m.EoS_p)),
                    eps=1e-6,
                )
            )
            / (B * m.EoS_p)
        )
m.dew_temp_liq = Expression(m.Comp, rule = _dew_temp_liq)

#-----------------------气相泡点温度---------------------------
def _bubble_temp_vap(m, j):
    def a(k):
        return (
            m.OmegaA * ((R * m.Tc[k]) ** 2 / m.Pc[k])
            * ((1 + m.fw[k] * (1 - sqrt(m.temperature_bubble / m.Tc[k]))) ** 2)
        )
    def b(k):
        return (m.OmegaB * R * m.Tc[k] / m.Pc[k])
        
    am = sum(
        sum(
            m._mole_frac_tbub[i]
            * m._mole_frac_tbub[j]
            * sqrt(a(i) * a(j))
            * (1 - m.kappa[i, j])
            for j in m.Comp
        )
        for i in m.Comp
    )
    bm = sum(m._mole_frac_tbub[i] * b(i) for i in m.Comp)

    A = am * m.P / (R * m.temperature_bubble) ** 2
    B = bm * m.P / (R * m.temperature_bubble)

    delta = (
        2 * sqrt(a(j)) / am
        * sum(m._mole_frac_tbub[i] * sqrt(a(i)) * (1 - m.kappa[j, i]) for i in m.Comp)
    )

    Z = m.crh.evaluate(args=((-(1+B-m.EoS_u*B)), 
                             (A-m.EoS_u*B-(m.EoS_u-m.EoS_w)*B**2), 
                             (-A*B-m.EoS_w*B**2-m.EoS_w*B**3)))

    return exp(
        (
            b(j) / bm * (Z - 1) * (B * m.EoS_p)
            - safe_log(Z - B, eps=1e-6) * (B * m.EoS_p)
            + A
            * (b(j) / bm - delta)
            * safe_log(
                (2 * Z + B * (m.EoS_u + m.EoS_p))
                / (2 * Z + B * (m.EoS_u - m.EoS_p)),
                eps=1e-6,
            )
        )
        / (B * m.EoS_p)
    )
m.bubble_temp_vap = Expression(m.Comp, rule = _bubble_temp_vap)
    
#-----------------------气相露点温度---------------------------
def _dew_temp_vap(m, j):
    def a(k):
        return (
            m.OmegaA * ((R * m.Tc[k]) ** 2 / m.Pc[k])
            * ((1 + m.fw[k] * (1 - sqrt(m.temperature_dew / m.Tc[k]))) ** 2)
        )
        
    def b(k):
        return (m.OmegaB * R * m.Tc[k] / m.Pc[k])

    am = sum(
        sum(
            m.mole_frac_comp[i]
            * m.mole_frac_comp[j]
            * sqrt(a(i) * a(j))
            * (1 - m.kappa[i, j])
            for j in m.Comp
        )
        for i in m.Comp
    )
    bm = sum(m.mole_frac_comp[i] * b(i) for i in m.Comp)

    A = am * m.P / (R * m.temperature_dew) ** 2
    B = bm * m.P / (R * m.temperature_dew)

    delta = (
        2 * sqrt(a(j)) / am
        * sum(m.mole_frac_comp[i] * sqrt(a(i)) * (1 - m.kappa[j, i]) for i in m.Comp)
    )

    Z = m.crh.evaluate(args=((-(1+B-m.EoS_u*B)), 
                             (A-m.EoS_u*B-(m.EoS_u-m.EoS_w)*B**2), 
                             (-A*B-m.EoS_w*B**2-m.EoS_w*B**3)))

    return exp(
        (
            b(j) / bm * (Z - 1) * (B * m.EoS_p)
            - safe_log(Z - B, eps=1e-6) * (B * m.EoS_p)
            + A
            * (b(j) / bm - delta)
            * safe_log(
                (2 * Z + B * (m.EoS_u + m.EoS_p))
                / (2 * Z + B * (m.EoS_u - m.EoS_p)),
                eps=1e-6,
            )
        )
        / (B * m.EoS_p)
    )
m.dew_temp_vap = Expression(m.Comp, rule = _dew_temp_vap)

######################
m._sum_mole_frac_tbub = Constraint(expr=1e3== \
                                   1e3 * sum(m._mole_frac_tbub[j] for j in m.Comp))
# ln(xi(T_dew))+ln(f_liq(T_dew)) = ln(xi)+ln(f_vap(T_bub))
def rule_bubble_temp(m, j):
    return log(m.mole_frac_comp[j]) + log(m.bubble_temp_liq[j]) == \
    log(m._mole_frac_tbub[j]) + log(m.bubble_temp_vap[j])
m.eq_temperature_bubble = Constraint(m.Comp, rule=rule_bubble_temp)

m._sum_mole_frac_tdew = Constraint(expr=1e3 == \
                                   1e3 * sum(m._mole_frac_tdew[j] for j in m.Comp))
# ln(xi)+ln(f_liq(T_bub)) = ln(xi(T_bub))+ln(f_vap(T_dew))
def rule_dew_temp(m, j):
    return log(m._mole_frac_tdew[j]) + log(m.dew_temp_liq[j]) == \
    log(m.mole_frac_comp[j]) + log(m.dew_temp_vap[j])
m.eq_temperature_dew = Constraint(m.Comp, rule=rule_dew_temp)
#######################

In [5]:
# 算出泡露点后，可以计算 T_eq 并带入计算了
# Definition of equilibrium temperature for smooth VLE
m._teq = Var(
    initialize=m.T.value,
    doc="Temperature for calculating phase equilibrium"
)
m._t1 = Var(
    initialize=m.T.value,
    doc="Intermediate temperature for calculating Teq"
)

m.eps_1 = Param(
    default=0.01,
    mutable=True,
    doc="Smoothing parameter for Teq"
)
m.eps_2 = Param(
    default=0.0005,
    mutable=True,
    doc="Smoothing parameter for Teq"
)

# PSE paper Eqn 13
def rule_t1(b):
    return b._t1 == 0.5 * (
        b.T + b.temperature_bubble
        + sqrt((b.T - b.temperature_bubble) ** 2 + b.eps_1**2)
    )
m._t1_constraint = Constraint(rule=rule_t1)

# PSE paper Eqn 14
# TODO : Add option for supercritical extension
def rule_teq(b):
    return b._teq == 0.5 * (
        b._t1 + b.temperature_dew
        - sqrt((b._t1 - b.temperature_dew) ** 2 + b.eps_2**2)
    )
m._teq_constraint = Constraint(rule=rule_teq)

In [6]:
def func_a(m, j):
    return (m.OmegaA * ((R * m.Tc[j]) ** 2 / m.Pc[j])
        * ((1 + m.fw[j] * (1 - sqrt(m._teq / m.Tc[j]))) ** 2))
m.a = Expression(m.Comp,rule=func_a)

def func_b(m, j):
    return (m.OmegaB * R * m.Tc[j] / m.Pc[j])
m.b = Expression(m.Comp,rule=func_b)

def rule_am(m, p):
    return sum(
    sum(
        m.mole_frac_phase_comp[p, i]
        * m.mole_frac_phase_comp[p, j]
        * sqrt(m.a[i] * m.a[j])
        * (1 - m.kappa[i, j])
        for j in m.Comp
    )
    for i in m.Comp
    )
m.am = Expression(m.Phase, rule=rule_am)

def rule_bm(m, p):
    return sum(m.mole_frac_phase_comp[p,i] * m.b[i] for i in m.Comp)
m.bm = Expression(m.Phase, rule=rule_bm)

def func_A(m,p):
    return m.am[p]*m.P/(R*m._teq)**2
m.A = Expression(m.Phase, rule=func_A)

def func_B(m,p):
    return m.bm[p]*m.P/(R*m._teq)
m.B = Expression(m.Phase, rule=func_B)

In [7]:
# 定义三次方程一般形式的系数
def rule_cubic_coef_b(m,p):
    return -1-m.B[p]+m.EoS_u*m.B[p]
m.cubic_coef_b = Expression(m.Phase,rule=rule_cubic_coef_b)

def rule_cubic_coef_c(m,p):
    return (m.A[p]-m.EoS_u*m.B[p]-(m.EoS_u-m.EoS_w)*m.B[p]**2)
m.cubic_coef_c = Expression(m.Phase,rule=rule_cubic_coef_c)

def rule_cubic_coef_d(m,p):
    return (-m.A[p]*m.B[p]-m.EoS_w*m.B[p]**2-m.EoS_w*m.B[p]**3)
m.cubic_coef_d = Expression(m.Phase,rule=rule_cubic_coef_d)

def evaluate_root(m,p):
    if p == 'Vap':
        return m.crh.evaluate(args=(m.cubic_coef_b[p], m.cubic_coef_c[p], m.cubic_coef_d[p]))
    else:
        return m.crl.evaluate(args=(m.cubic_coef_b[p], m.cubic_coef_c[p], m.cubic_coef_d[p]))
m.external_expr = Expression(m.Phase, rule=evaluate_root)

m.Z = Var(m.Phase, initialize = 1)
def rule_cubic_Z(m,p):
    return m.Z[p] == m.external_expr[p]
m.cubic_z = Constraint(m.Phase, rule = rule_cubic_Z)

In [8]:
# 用压缩因子计算相平衡
def rule_delta(m, p, i):
    # See pg. 145 in Properties of Gases and Liquids
    return (2 * sqrt(m.a[i]) / m.am[p]
        * sum(
            m.mole_frac_phase_comp[p, j]
            * sqrt(m.a[j])
            * (1 - m.kappa[i, j])
            for j in m.Comp
        )
    )
m.delta = Expression(m.Phase, m.Comp, rule=rule_delta)

def ln_fug_coeff_cubic(b, p, j):
    # See pg. 145 in Properties of Gases and Liquids
    return (
            b.b[j] / b.bm[p] * (b.Z[p] - 1) * (b.B[p] * b.EoS_p)
            - safe_log(b.Z[p] - b.B[p], eps=1e-6)
            * (m.B[p] * m.EoS_p)
            + b.A[p]
            * (b.b[j] / b.bm[p] - b.delta[p, j])
            * safe_log(
                (2 * b.Z[p] + b.B[p] * (b.EoS_u + b.EoS_p))
                / (2 * b.Z[p] + b.B[p] * (b.EoS_u - b.EoS_p)),
                eps=1e-6,
            )
        ) / (b.B[p] * b.EoS_p)
m.ln_fug_coeff = Expression(m.Phase, m.Comp, rule = ln_fug_coeff_cubic)

In [9]:
# 相平衡约束
def rule_VLE(m,c):
    return m.ln_fug_coeff['Vap',c] + safe_log(m.mole_frac_phase_comp['Vap', c],eps=1e-6) == \
    m.ln_fug_coeff['Liq',c] + safe_log(m.mole_frac_phase_comp['Liq', c],eps=1e-6)
m.rule_equilibrium = Constraint(m.Comp, rule = rule_VLE)

In [34]:
m.flow_mol.fix(100)
m.mole_frac_comp["benzene"].fix(0.5)
m.mole_frac_comp["toluene"].fix(0.5)
m.T.fix(368)
m.P.fix(1e5)

In [35]:
degrees_of_freedom(m)

0

In [36]:
Initialization(m)

In [37]:
m.temperature_bubble.value

365.1935490433224

In [38]:
m.temperature_dew.value

371.862749804761

In [39]:
m._teq.value

368

In [40]:
exp(value(m.ln_fug_coeff['Liq','benzene'])) #1.492049

1.5752082204689077

In [41]:
exp(value(m.ln_fug_coeff['Liq','toluene'])) #0.621563

0.6998669335076699

In [42]:
exp(value(m.ln_fug_coeff['Vap','benzene'])) #0.97469

0.9772795580644094

In [43]:
exp(value(m.ln_fug_coeff['Vap','toluene'])) #0.964642

0.9613144576210214

In [44]:
solver = SolverFactory('ipopt')
solver.options['tol'] = 1e-6
results = solver.solve(m)

In [45]:
for p in m.Phase:
    for c in m.Comp:
        print(value(m.Z[p]))

0.003983679341526769
0.003983679341526769
0.9441927576897298
0.9441927576897298


In [46]:
m.flow_mol_phase.pprint()

flow_mol_phase : Size=2, Index=Phase
    Key : Lower : Value             : Upper : Fixed : Stale : Domain
    Liq :     0 : 13.60665269249073 :  None : False : False : NonNegativeReals
    Vap :     0 : 86.39334730750927 :  None : False : False : NonNegativeReals


In [47]:
m.mole_frac_phase_comp.pprint()

mole_frac_phase_comp : Size=4, Index=mole_frac_phase_comp_index
    Key                : Lower : Value              : Upper : Fixed : Stale : Domain
    ('Liq', 'benzene') :     0 : 0.3323157889862633 :  None : False : False :  Reals
    ('Liq', 'toluene') :     0 : 0.6676842110137368 :  None : False : False :  Reals
    ('Vap', 'benzene') :     0 : 0.5264096819068848 :  None : False : False :  Reals
    ('Vap', 'toluene') :     0 : 0.4735903180931153 :  None : False : False :  Reals


In [24]:
m.mole_frac_comp.pprint()

mole_frac_comp : Size=2, Index=Comp
    Key     : Lower : Value : Upper : Fixed : Stale : Domain
    benzene :     0 :   0.5 :  None :  True :  True :  Reals
    toluene :     0 :   0.5 :  None :  True :  True :  Reals


In [25]:
m.temperature_bubble.value

364.63713679370744

In [26]:
m.temperature_dew.value

370.1305970426683

In [31]:
m._teq.value

368.0000074047902